## Import Dependencies

In [93]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from tqdm import tqdm

from Bio import pairwise2
from Bio.Seq import Seq
from Bio.pairwise2 import format_alignment

from itertools import combinations

import numpy as np

## 1 Filter The Original Dataset Set To Only Contain HIV type 1 Only

### 1.1 Read CSV

In [53]:
df = pd.read_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded.csv")
df.head()

C:\Users\M S I\AppData\Local\Temp\ipykernel_38836\3386473262.py:1: DtypeWarning: Columns (7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded.csv")


,C_ID,Name,Common Name,Scientific Name,Quantity,Non plant ingredient,Action,Application,Comments,Bacteria,...,Mw,CAS RN,InChIKey,InChICode,SMILES,target_name,species,bdb_id,ligand_smiles,pdb_link
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,346.141638,77-06-5,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Protease,(Human immunodeficiency virus type 1),BDBM50478896,C[C@H]1[C@@H]2CC[C@](C)(O)[C@@H]3[C@H]4[C@@H]5...,https://www.rcsb.org/search?request=%7B%22quer...
1,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,346.141638,77-06-5,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Nuclear factor NF-kappa-B p105 subunit,(Human),BDBM50561592,C[C@@]12[C@H]3[C@H](C(O)=O)[C@@]45CC(=C)[C@@](...,https://www.rcsb.org/search?request=%7B%22quer...
2,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,346.141638,77-06-5,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Transcription factor p65,(Human),BDBM50561592,C[C@@]12[C@H]3[C@H](C(O)=O)[C@@]45CC(=C)[C@@](...,https://www.rcsb.org/search?request=%7B%22quer...
3,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Tyrosine-protein phosphatase non-receptor type...,(Human),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...
4,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Tyrosine-protein phosphatase non-receptor type...,(Human),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...


### 1.2 Get To Know The Unique Values From "Species" Column

In [54]:
df["species"].unique()

array(['(Human immunodeficiency virus type 1)', '(Human)', '(HIV-1)',
       '(Sheep)', '(Rabbit)', '(Bacillus licheniformis)', '(Chicken)',
       '(Horse)', '(Pacific electric ray)', '(Almond)', '(Mouse)',
       '(Rat)', '(White button mushroom)',
       '(Escherichia coli (strain K12))',
       '(Streptococcus mutans serotype c (strain ATCC 70061...)',
       '(Yellow koji mold)', '(SARS-CoV)', '(Clostridium botulinum)',
       '(Dengue virus)', '(Leishmania major)', '(Pig)', '(Bovine)',
       '(Mouse-ear cress)', '(Plasmodium falciparum (isolate 3D7))',
       '(Streptococcus pneumoniae D39)',
       '(Streptococcus pyogenes M1 GAS)', '(Guinea pig)',
       '(Chinese hamster)',
       '(Mycobacterium tuberculosis (strain ATCC 25618 / H3...)',
       '(Toxoplasma gondii)', '(Trypanosoma brucei)', '(Silk moth)',
       '(Trypanosoma brucei brucei (strain 927/4 GUTat10.1))',
       '(Wild turkey)',
       '(Hepatitis C virus genotype 1b (isolate Con1) (HCV))', '(WNV)',
       '(Esch

### 1. 3 Filter The Original Dataset => Species = Human Immunodeficiency Virus type 1

In [55]:
df_filtered = df.loc[df["species"] == "(Human immunodeficiency virus type 1)"]
df_filtered.head()

,C_ID,Name,Common Name,Scientific Name,Quantity,Non plant ingredient,Action,Application,Comments,Bacteria,...,Mw,CAS RN,InChIKey,InChICode,SMILES,target_name,species,bdb_id,ligand_smiles,pdb_link
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,346.141638,77-06-5,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Protease,(Human immunodeficiency virus type 1),BDBM50478896,C[C@H]1[C@@H]2CC[C@](C)(O)[C@@H]3[C@H]4[C@@H]5...,https://www.rcsb.org/search?request=%7B%22quer...
7,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Gag-Pol polyprotein [489-587],(Human immunodeficiency virus type 1),BDBM50613450,CC(=C)[C@H]1CC[C@@]2(C)[C@H]1CC[C@]1(C)[C@@H]2...,https://www.rcsb.org/search?request=%7B%22quer...
24,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Glycogen phosphorylase,(Human immunodeficiency virus type 1),BDBM50050957,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...
25,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,muscle form,(Human immunodeficiency virus type 1),BDBM50050952,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...
26,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,332.162374,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Protein kinase C epsilon type,(Human immunodeficiency virus type 1),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...


In [5]:
df_filtered["C_ID"].unique()
len(df_filtered)

9050

### 1.4 Group C_ID by bdb_id

In [208]:
group_dict = {}

for _, row in df_filtered.iterrows():
    bdb = row["bdb_id"]
    c_id = row["C_ID"]
    
    if bdb not in group_dict:
        group_dict[bdb] = set()  
        
    group_dict[bdb].add(c_id)
    
bdb_grouped_df = pd.DataFrame({
    "bdb_id": group_dict.keys(),
    "C_IDs": [", ".join(map(str, c_ids)) for c_ids in group_dict.values()]
})

bdb_grouped_df.to_csv("./Data HIV Type 1/bdb_grouped.csv")

bdb_grouped_df.head()

,bdb_id,C_IDs
0,BDBM50478896,"C00057476, C00000003,, C00060216, C00000259,"
1,BDBM50613450,"C00000004,, C00000051,, C00000020,, C00000009,"
2,BDBM50050957,"C00000020,, C00000029,, C00000044,, C00000027,..."
3,BDBM50050952,"C00000020,, C00000029,, C00000027,, C00000004,..."
4,BDBM50050955,"C00000020,, C00000029,, C00000027,, C00000004,..."


## 2. Scrapping Protein Code From pdb_link

### 2.1 Scrapping HIV Protein Code

In [ ]:
driver = webdriver.Chrome()

df_filtered["query"] = None

for index, row in tqdm(df_filtered.iterrows(), total=df_filtered.shape[0], desc="Processing", unit="row"):
    link = row["pdb_link"]
    driver.get(link)

    try:
        input_element = driver.find_element(By.CSS_SELECTOR, "input[type='text'][placeholder='Enter one or more search terms.']")

        if input_element:
            value = input_element.get_attribute('value')
            df_filtered.at[index, 'query'] = value
        else:
            print('Input element tidak ditemukan.')

    except Exception as e:
        print(f'Terjadi kesalahan: {e}')

driver.quit()

print(df_filtered.head())

In [11]:
df_filtered.to_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded-hiv-only-n-query.csv")

### 2.2 Get Unique HIV Protein Code

In [209]:
df_filtered = pd.read_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded-hiv-only-n-query.csv")
df_filtered = df_filtered.drop("Unnamed: 0", axis=1)
df_filtered.head()

,C_ID,Name,Common Name,Scientific Name,Quantity,Non plant ingredient,Action,Application,Comments,Bacteria,...,CAS RN,InChIKey,InChICode,SMILES,target_name,species,bdb_id,ligand_smiles,pdb_link,query
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,77-06-5,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Protease,(Human immunodeficiency virus type 1),BDBM50478896,C[C@H]1[C@@H]2CC[C@](C)(O)[C@@H]3[C@H]4[C@@H]5...,https://www.rcsb.org/search?request=%7B%22quer...,"9HVP,7YF6,7WCQ,7WBS,6OYR,6OYD,6OGP,6MCS,6MCR,5..."
1,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Gag-Pol polyprotein [489-587],(Human immunodeficiency virus type 1),BDBM50613450,CC(=C)[C@H]1CC[C@@]2(C)[C@H]1CC[C@]1(C)[C@@H]2...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1..."
2,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Glycogen phosphorylase,(Human immunodeficiency virus type 1),BDBM50050957,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1..."
3,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,muscle form,(Human immunodeficiency virus type 1),BDBM50050952,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1..."
4,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,468-44-0,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Protein kinase C epsilon type,(Human immunodeficiency virus type 1),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1..."


In [211]:
query_data = df_filtered["query"].tolist()
query_list = []

for query in query_data:
    query_list.extend(query.split(","))
    
unique_query_list = list(set(query_list))
df_unique_hiv_protein_list = pd.DataFrame(unique_query_list, columns=["hiv protein"])
df_unique_hiv_protein_list.head()

,hiv protein
0,2R43
1,1TVR
2,2ZE2
3,6PUY
4,1MES


In [212]:
df_unique_hiv_protein_list.to_csv("./Data HIV Type 1/unique_query_hiv.csv", index=False)

## 3. Scrapping HIV Protein Fasta

### 3.1 Read Unique HIV Protein Code DF

In [213]:
df_unique_hiv_protein_list = pd.read_csv("./Data HIV Type 1/unique_query_hiv.csv")
df_unique_hiv_protein_list.head()

,hiv protein
0,2R43
1,1TVR
2,2ZE2
3,6PUY
4,1MES


### 3.2 Add Title Column To Unique HIV Protein DF

In [17]:
driver = webdriver.Chrome()

df_unique_hiv_protein_list["title"] = None

for index, row in tqdm(df_unique_hiv_protein_list.iterrows(), total=df_unique_hiv_protein_list.shape[0], desc="Processing", unit="row"):
    link = f"https://pdbj.org/mine/structural_details/{row["query"]}"
    
    driver.get(link)

    try:
        title_element = driver.find_element(By.CSS_SELECTOR, "#PDBExplorerPlane h2")

        if title_element:
            title_value = title_element.text
            df_unique_hiv_protein_list.at[index, 'title'] = title_value
        else:
            print('Input element tidak ditemukan.')

    except Exception as e:
        print(f'Terjadi kesalahan: {e}')
        
driver.quit()

df_unique_hiv_protein_list.to_excel("./Data HIV Type 1/hiv_data.xlsx")
df_unique_hiv_protein_list.head()

Processing: 100%|██████████| 984/984 [02:58<00:00,  5.51row/s]


,query,title
0,1EC2,HIV-1 protease in complex with the inhibitor B...
1,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE
2,7KE0,HIV-1 Integrase catalytic core domain complexe...
3,4CER,Interrogating HIV integrase for compounds that...
4,2BPY,HIV-1 protease-inhibitor complex


### 3.3 Download Fasta File of HIV Protein

In [18]:
import requests
import os

save_folder = "./Data HIV Type 1/hiv_fasta_files"
os.makedirs(save_folder, exist_ok=True)

counter = 0

for index, row in tqdm(df_unique_hiv_protein_list.iterrows(), total=df_unique_hiv_protein_list.shape[0], desc="Downloading FASTA Files"):
    hiv_protein_code = row["hiv protein"]
    url = f"https://pdbj.org/rest/newweb/fetch/file?cat=pdb&type=fasta&id={hiv_protein_code}"
    
    file_path = os.path.join(save_folder, f"{hiv_protein_code}.tsv")

    try:
        response = requests.get(url, stream=True)

        if response.status_code == 200:
            with open(file_path, "wb") as file:
                for chunk in response.iter_content(chunk_size=1024):  
                    file.write(chunk)
            counter += 1
        else:
            print(f"❌ Gagal mendownload {hiv_protein_code} (Status Code: {response.status_code})")

    except Exception as e:
        print(f"❌ Terjadi kesalahan saat mendownload {hiv_protein_code}: {e}")

print(f"✅ file sebanyak {counter} berhasil disimpan.")

✅ file sebanyak 984 berhasil disimpan.


### 3.4 Add Sub Seq of HIV Protein from Fasta File - Extract New DF 

In [19]:
data = []

for index, row in tqdm(df_unique_hiv_protein_list.iterrows(), total=df_unique_hiv_protein_list.shape[0], desc="Processing", unit="row"):
    query_id = row["query"]
    file_path = f"./Data HIV Type/hiv_fasta_files/{query_id}.tsv"

    with open(file_path, "r") as file:
        lines = file.readlines()

    current_query = query_id
    current_title = row["title"]
    current_seq_code = None
    current_seq_title = ""
    current_sequence = ""

    for line in lines:
        line = line.strip()
        if line.startswith(">"):  
            if current_seq_code:  
                data.append([current_query, current_title, current_seq_code, current_seq_title, current_sequence])
            current_seq_code = line[1:7]  
            current_seq_title = line[8:]
            current_sequence = ""
        else:
            current_sequence += line  

    if current_seq_code:
        data.append([current_query, current_title, current_seq_code, current_seq_title, current_sequence])

# Konversi ke DataFrame
df_fasta = pd.DataFrame(data, columns=["query", "title", "seq_code", "seq_title", "seq"])

Processing: 100%|██████████| 984/984 [00:05<00:00, 168.83row/s]


In [20]:
df_fasta.head()

,query,title,seq_code,seq_title,seq
0,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_A,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...
3,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_B,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...
4,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_C,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...


In [21]:
df_fasta.to_excel("./Data HIV Type 1/hiv_data.xlsx", index=False)

## 4. Filter HIV Protein Data - HIV Protease, HIV Reverse Transcriptase (RT), HIV Envelope Glycoproteins, HIV Capsid

1. HIV Protease

Find keyword: 'protease'

2. HIV Reverse Transcriptase (RT)

Find keyword: 'transcriptase'

3. HIV Envelope Glycoproteins

Find keyword: 'glycoprotein'

4. HIV Capsid

Find keyword: 'capsid'

### 4.1 Read HIV Data

In [481]:
hiv_df = pd.read_excel("./Data HIV Type 1/hiv_data.xlsx")
hiv_df.head()

,query,title,seq_code,seq_title,seq
0,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_A,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...
3,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_B,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...
4,1BL3,CATALYTIC DOMAIN OF HIV-1 INTEGRASE,1bl3_C,INTEGRASE,MHGQVDCSPGIWQLDCTHLEGKVILVAVHVASGYIEAEVIPAETGQ...


### 4.2 Get To Know Unique Protein Title

In [66]:
seq_title = hiv_df["seq_title"]
seq_title.unique()

array([' HIV-1 PROTEASE', ' INTEGRASE', ' Integrase', ' Protease',
       ' Reverse Transcriptase', ' HIV-1 REVERSE TRANSCRIPTASE',
       ' REVERSE TRANSCRIPTASE/RIBONUCLEASE H', ' P51 RT',
       ' p66 Reverse transcriptase/RNaseH',
       ' p51 Reverse transcriptase/RNaseH', ' Primer DNA',
       ' Template DNA', ' Pol polyprotein',
       ' Chimeric Sso7d and HIV-1 integrase',
       ' viral DNA non-transferred strand',
       ' viral DNA transferred strand', ' protease',
       ' Reverse transcriptase/ribonuclease H', ' p51 RT',
       ' HIV-1 ENVELOPE GLYCOPROTEIN GP160', ' IMMUNOGLOBULIN G PGT151',
       ' p66 RT', ' reverse transcriptase p51 subunit',
       ' HIV-1 viral RNA fragment',
       ' tRNA lysine 3 fragment with GNRA tetraloop', ' HIV-1 Protease',
       ' Integrase catalytic',
       ' Reverse transcriptase/ribonuclease H, p66 subunit',
       ' Reverse transcriptase/ribonuclease H, p51 subunit',
       ' HIV INTEGRASE', ' HIV-1 reverse transcriptase, p66 subunit',

### 4.3 Filter HIV Data Set To Only Contain HIV Protease, HIV Reverse Transcriptase (RT), HIV Envelope Glycoproteins, HIV Capsid

In [482]:
keywords = ["protease", "transcriptase", "glycoprotein", "capsid"]
pattern = "|".join(keywords)

hiv_df_filtered = hiv_df[hiv_df["seq_title"].str.contains(pattern, case=False, na=False)]
hiv_df_filtered.head()

,query,title,seq_code,seq_title,seq
0,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
8,2BPY,HIV-1 protease-inhibitor complex,2bpy_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
9,2BPY,HIV-1 protease-inhibitor complex,2bpy_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
10,7MA7,HIV-1 Protease (I84V) in Complex with UMass7,7ma7_A,Protease,PQITLWKRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...


In [193]:
# List seq_code yang ingin dikelompokkan
seq_code_list = ["1rtd_B", "1rtd_D"]  # contoh, gunakan seluruh daftar Anda

# Filter dataframe berdasarkan seq_code yang ada dalam list
filtered_df = hiv_df_filtered[hiv_df_filtered["seq_code"].isin(seq_code_list)]

# Buat mapping seq_code ke kategori berdasarkan seq_title
seq_code_to_category = dict(zip(filtered_df["seq_code"], filtered_df["seq_title"]))

# Cetak hasil kelompokannya
for seq_code in seq_code_list:
    category = seq_code_to_category.get(seq_code, "Unknown")  # Jika tidak ditemukan, beri label "Unknown"
    print(f"{seq_code}: {category}")

category_df = pd.DataFrame(seq_code_to_category.items(), columns=["seq_code", "category"])
category_df.head()


1rtd_B:  DNA PRIMER FOR REVERSE TRANSCRIPTASE
1rtd_D:  DNA PRIMER FOR REVERSE TRANSCRIPTASE


,seq_code,category
0,1rtd_B,DNA PRIMER FOR REVERSE TRANSCRIPTASE
1,1rtd_D,DNA PRIMER FOR REVERSE TRANSCRIPTASE


In [7]:
hiv_df_filtered["seq_title"].unique()

array([' HIV-1 PROTEASE', ' Protease', ' Reverse Transcriptase',
       ' HIV-1 REVERSE TRANSCRIPTASE',
       ' REVERSE TRANSCRIPTASE/RIBONUCLEASE H',
       ' p66 Reverse transcriptase/RNaseH',
       ' p51 Reverse transcriptase/RNaseH', ' protease',
       ' Reverse transcriptase/ribonuclease H',
       ' HIV-1 ENVELOPE GLYCOPROTEIN GP160',
       ' reverse transcriptase p51 subunit', ' HIV-1 Protease',
       ' Reverse transcriptase/ribonuclease H, p66 subunit',
       ' Reverse transcriptase/ribonuclease H, p51 subunit',
       ' HIV-1 reverse transcriptase, p66 subunit',
       ' HIV-1 reverse transcriptase, p51 subunit',
       ' Envelope glycoprotein gp120', ' Envelope glycoprotein gp41',
       ' HIV-1 Reverse Transcriptase P66 subunit',
       ' HIV-1 Reverse Transcriptase P51 subunit',
       ' reverse transcriptase p66 subunit',
       ' HIV-1 REVERSE TRANSCRIPTASE (SUBUNIT P66)',
       ' HIV-1 REVERSE TRANSCRIPTASE (SUBUNIT P51)', ' HIV-1 protease',
       ' PROTEASE RETR

In [8]:
hiv_df_filtered.to_excel("./Data HIV Type 1/hiv_data_filtered.xlsx", index=False)

In [16]:
hiv_df_filtered = pd.read_excel("./Data HIV Type 1/hiv_data_filtered.xlsx")
hiv_df_filtered.head()

,query,title,seq_code,seq_title,seq
0,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
2,2BPY,HIV-1 protease-inhibitor complex,2bpy_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
3,2BPY,HIV-1 protease-inhibitor complex,2bpy_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
4,7MA7,HIV-1 Protease (I84V) in Complex with UMass7,7ma7_A,Protease,PQITLWKRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...


## 5. Do Seq Matching

### 5.1 Create DF For Seq Matching Result

In [9]:
seq_matching_df = comparison_result = pd.DataFrame(columns=['Seq Name 1', 'Seq Name 2', 'Seq 1', 'Seq 2', 'Alignment Score', 'Total Alignment', 'Similarities'])
seq_matching_df.head()

,Seq Name 1,Seq Name 2,Seq 1,Seq 2,Alignment Score,Total Alignment,Similarities


### 5.2 Seq Matching Process

In [ ]:
output_file = "./Data HIV Type 1/seq_matching_result.csv"

# Tulis header (jika file belum ada)
with open(output_file, 'w') as f:
    f.write("seq_code1,seq_code2,seq1,seq2,alignment_score,total_alignment,similarities\n")

# Loop melalui kombinasi unik untuk menghindari duplikasi perbandingan
for _, protein1 in tqdm(hiv_df_filtered.iterrows(), total=len(hiv_df_filtered), desc="Comparing Progress"):
    for _, protein2 in hiv_df_filtered.iterrows():
    
        seq1 = Seq(protein1.seq)
        seq2 = Seq(protein2.seq)

        alignments = pairwise2.align.globalxx(seq1, seq2)

        alignment_score = alignments[0][2]
        total_alignment = alignments[0][4]
        similarities = round(alignment_score / total_alignment * 100, 2)

        # Simpan hasil langsung ke CSV
        with open(output_file, 'a') as f:
            f.write(f"{protein1.seq_code},{protein2.seq_code},{protein1.seq},{protein2.seq},{alignment_score},{total_alignment},{similarities}\n")

print(f"Proses selesai! Hasil disimpan di {output_file}")


In [ ]:
import pandas as pd
from Bio import pairwise2
from Bio.Seq import Seq
from itertools import product
from tqdm import tqdm
from joblib import Parallel, delayed
import os

output_file = "./Data HIV Type 1/seq_matching_result.csv"

if not os.path.exists(output_file):
    with open(output_file, 'w') as f:
        f.write("seq_code1,seq_code2,seq1,seq2,alignment_score,total_alignment,similarities\n")

def compute_alignment(protein1, protein2):
    seq1 = Seq(protein1.seq)
    seq2 = Seq(protein2.seq)

    alignments = pairwise2.align.globalxx(seq1, seq2)
    alignment_score = alignments[0][2]
    total_alignment = alignments[0][4]
    similarities = round(alignment_score / total_alignment * 100, 2)

    result = f"{protein1.seq_code},{protein2.seq_code},{protein1.seq},{protein2.seq},{alignment_score},{total_alignment},{similarities}\n"

    with open(output_file, 'a') as f:
        f.write(result)

protein_list = list(hiv_df_filtered.itertuples(index=False))

protein_pairs = list(product(protein_list, repeat=2))

Parallel(n_jobs=-1, backend="loky")(delayed(compute_alignment)(p1, p2) for p1, p2 in tqdm(protein_pairs, desc="Comparing Progress"))

print(f"Proses selesai! Hasil disimpan di {output_file}")

## 6. Filter Seq Matching Result with > 50% Similarities

### 6.1 Read Seq Matching Result File

In [483]:
df_seq_matching_filtered = pd.read_csv("./Data HIV Type 1/seq_matching_result.csv")
df_seq_matching_filtered.head()

,seq_code1,seq_code2,seq1,seq2,alignment_score,total_alignment,similarities
0,1ec2_A,1ec2_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
1,1ec2_A,1ec2_B,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
2,1ec2_A,2bpy_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
3,1ec2_A,2bpy_B,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
4,1ec2_A,7ma7_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWKRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,96.0,102,94.12


### 6.2 Filter Seq Matching Result - Similarities > 50%

In [160]:
df_seq_matching_filtered = df_seq_matching_filtered.loc[df_seq_matching_filtered['similarities'] > 50]
df_seq_matching_filtered.head()

,seq_code1,seq_code2,seq1,seq2,alignment_score,total_alignment,similarities
0,1ec2_A,1ec2_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
1,1ec2_A,1ec2_B,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
2,1ec2_A,2bpy_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
3,1ec2_A,2bpy_B,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,99.0,99,100.00
4,1ec2_A,7ma7_A,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...,PQITLWKRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...,96.0,102,94.12


### 6.3 Solving Problem - Too Much Data

#### 6.3.1 Protease DF

In [465]:
protease_df = hiv_df[hiv_df["seq_title"].str.contains("protease", case=False, na=False)]
protease_df.head()

,query,title,seq_code,seq_title,seq
0,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
1,1EC2,HIV-1 protease in complex with the inhibitor B...,1ec2_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
8,2BPY,HIV-1 protease-inhibitor complex,2bpy_A,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
9,2BPY,HIV-1 protease-inhibitor complex,2bpy_B,HIV-1 PROTEASE,PQITLWQRPLVTIKIGGQLKEALLDTGADDTVLEEMSLPGRWKPKM...
10,7MA7,HIV-1 Protease (I84V) in Complex with UMass7,7ma7_A,Protease,PQITLWKRPLVTIKIGGQLKEALLDTGADDTVLEEMNLPGRWKPKM...


In [466]:
protease_count_df = protease_df.groupby('query').agg(
    count=('seq_code', 'count'),
    seq_codes=('seq_code', lambda x: list(x))
).reset_index()

protease_count_sample = protease_count_df.sample(n=50, random_state=42)  # random_state untuk replikasi

# Menampilkan hasil
protease_count_sample.head()

,query,count,seq_codes
33,1GNO,2,"[1gno_A, 1gno_B]"
108,2QHC,2,"[2qhc_A, 2qhc_B]"
240,7M9O,2,"[7m9o_A, 7m9o_B]"
259,7MA8,2,"[7ma8_A, 7ma8_B]"
154,5HVP,2,"[5hvp_A, 5hvp_B]"


In [467]:
protease_seq = np.concatenate(protease_count_sample['seq_codes'].values)
len(protease_seq)

98

#### 6.3.2 Transcriptase DF

In [468]:
transcriptase_df = hiv_df[hiv_df["seq_title"].str.contains("transcriptase", case=False, na=False)]
transcriptase_df.head()

,query,title,seq_code,seq_title,seq
12,3T19,Crystal structure of HIV-1 reverse transcripta...,3t19_A,Reverse Transcriptase,MNSPISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEK...
13,3T19,Crystal structure of HIV-1 reverse transcripta...,3t19_B,Reverse Transcriptase,MNSPISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEK...
14,1RTJ,MECHANISM OF INHIBITION OF HIV-1 REVERSE TRANS...,1rtj_A,HIV-1 REVERSE TRANSCRIPTASE,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...
15,1RTJ,MECHANISM OF INHIBITION OF HIV-1 REVERSE TRANS...,1rtj_B,HIV-1 REVERSE TRANSCRIPTASE,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...
16,2VG5,Crystal structures of HIV-1 reverse transcript...,2vg5_A,REVERSE TRANSCRIPTASE/RIBONUCLEASE H,PISPIETVPVKLKPGMDGPKVKQWPLTEEKIKALVEICTEMEKEGK...


In [470]:
transcriptase_count_df = transcriptase_df.groupby('query').agg(
    count=('seq_code', 'count'),
    seq_codes=('seq_code', lambda x: list(x))
).reset_index()

transcriptase_count_sample = transcriptase_count_df.sample(n=50, random_state=42)  # random_state untuk replikasi

# Menampilkan hasil
transcriptase_count_sample.head()

,query,count,seq_codes
287,7KWU,2,"[7kwu_A, 7kwu_B]"
329,8DXE,1,[8dxe_A]
323,7Z2G,2,"[7z2g_A, 7z2g_B]"
145,4B3Q,1,[4b3q_A]
55,1UWB,2,"[1uwb_A, 1uwb_B]"


In [471]:
transcriptase_seq = np.concatenate(transcriptase_count_sample['seq_codes'].values)
len(transcriptase_seq)

101

#### 6.3.4 Glycoprotein DF

In [472]:
glycoprotein_df = hiv_df[hiv_df["seq_title"].str.contains("glycoprotein", case=False, na=False)]
glycoprotein_df.head()

,query,title,seq_code,seq_title,seq
38,5FUU,Ectodomain of cleaved wild type JR-FL EnvdCT t...,5fuu_A,HIV-1 ENVELOPE GLYCOPROTEIN GP160,TEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVP...
39,5FUU,Ectodomain of cleaved wild type JR-FL EnvdCT t...,5fuu_B,HIV-1 ENVELOPE GLYCOPROTEIN GP160,AVGIGAVFLGFLGAAGSTMGAASMTLTVQARLLLSGIVQQQNNLLR...
40,5FUU,Ectodomain of cleaved wild type JR-FL EnvdCT t...,5fuu_C,HIV-1 ENVELOPE GLYCOPROTEIN GP160,TEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVP...
41,5FUU,Ectodomain of cleaved wild type JR-FL EnvdCT t...,5fuu_D,HIV-1 ENVELOPE GLYCOPROTEIN GP160,AVGIGAVFLGFLGAAGSTMGAASMTLTVQARLLLSGIVQQQNNLLR...
42,5FUU,Ectodomain of cleaved wild type JR-FL EnvdCT t...,5fuu_E,HIV-1 ENVELOPE GLYCOPROTEIN GP160,TEKLWVTVYYGVPVWKEATTTLFCASDAKAYDTEVHNVWATHACVP...


In [474]:
glycoprotein_count_df = glycoprotein_df.groupby('query').agg(
    count=('seq_code', 'count'),
    seq_codes=('seq_code', lambda x: list(x))
).reset_index()

glycoprotein_count_sample = glycoprotein_count_df.sample(n=20, random_state=42)  # random_state untuk replikasi

# Menampilkan hasil
glycoprotein_count_sample.head()

,query,count,seq_codes
27,8FAE,6,"[8fae_A, 8fae_B, 8fae_C, 8fae_D, 8fae_E, 8fae_F]"
15,6OPQ,9,"[6opq_A, 6opq_C, 6opq_D, 6opq_F, 6opq_H, 6opq_..."
23,7Z3A,6,"[7z3a_A, 7z3a_B, 7z3a_C, 7z3a_D, 7z3a_E, 7z3a_F]"
17,6VPX,6,"[6vpx_A, 6vpx_B, 6vpx_C, 6vpx_D, 6vpx_E, 6vpx_F]"
8,6MUG,2,"[6mug_A, 6mug_D]"


In [475]:
glycoprotein_seq = np.concatenate(glycoprotein_count_sample['seq_codes'].values)
len(glycoprotein_seq)

126

#### 6.3.5 Capsid DF

In [476]:
capsid_df = hiv_df[hiv_df["seq_title"].str.contains("capsid", case=False, na=False)]
capsid_df.head()

,query,title,seq_code,seq_title,seq
915,7URN,Structure of HIV-1 capsid declination,7urn_A,HIV-1 capsid protein,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...
916,7URN,Structure of HIV-1 capsid declination,7urn_B,HIV-1 capsid protein,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...
917,7URN,Structure of HIV-1 capsid declination,7urn_C,HIV-1 capsid protein,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...
918,7URN,Structure of HIV-1 capsid declination,7urn_D,HIV-1 capsid protein,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...
919,7URN,Structure of HIV-1 capsid declination,7urn_E,HIV-1 capsid protein,PIVQNLQGQMVHQAISPRTLNAWVKVVEEKAFSPEVIPMFSALSEG...


In [478]:
capsid_count_df = capsid_df.groupby('query').agg(
    count=('seq_code', 'count'),
    seq_codes=('seq_code', lambda x: list(x))
).reset_index()

capsid_count_df = capsid_count_df[capsid_count_df['query'] != "6X63"]

capsid_count_df.head()

,query,count,seq_codes
0,5O2U,2,"[5o2u_A, 5o2u_C]"
1,6SKK,6,"[6skk_A, 6skk_B, 6skk_C, 6skk_D, 6skk_E, 6skk_F]"
3,7M9F,1,[7m9f_A]
4,7URN,7,"[7urn_A, 7urn_B, 7urn_C, 7urn_D, 7urn_E, 7urn_..."
5,8EJL,4,"[8ejl_A, 8ejl_B, 8ejl_C, 8ejl_F]"


In [479]:
capsid_seq = np.concatenate(capsid_count_df['seq_codes'].values)
len(capsid_seq)

20

#### 6.3.6 List All Sub Seq Protein Code from Protease, Transcriptase, Glycoprotei, Capsid

In [480]:
seq = np.concatenate([protease_seq, transcriptase_seq, glycoprotein_seq, capsid_seq])
len(seq)

345

#### 6.3.7 Filter Seq Mathching Filtered by "seq" list

In [161]:
df_seq_matching_filtered = df_seq_matching_filtered[df_seq_matching_filtered['seq_code1'].isin(seq)]

In [162]:
df_seq_matching_filtered = df_seq_matching_filtered[df_seq_matching_filtered['seq_code2'].isin(seq)]

### 6.4 Tidying Up Seq Matching Filtered DF

#### 6.4.1 Take seq_code1 and seq_code2 Column Only

In [163]:
df_seq_matching_filtered = df_seq_matching_filtered.loc[:, "seq_code1":"seq_code2"]
df_seq_matching_filtered.head()

,seq_code1,seq_code2
0,1ec2_A,1ec2_A
1,1ec2_A,1ec2_B
40,1ec2_A,7wcq_A
58,1ec2_A,1hpv_A
59,1ec2_A,1hpv_B


#### 6.4.2 Change Column Name seq_code1 to Protein 1 and seq_code2 to Protein 2

In [164]:
df_seq_matching_filtered.rename(columns={'seq_code1': 'Protein 1', 'seq_code2': 'Protein 2'}, inplace=True)
df_seq_matching_filtered.head()

,Protein 1,Protein 2
0,1ec2_A,1ec2_A
1,1ec2_A,1ec2_B
40,1ec2_A,7wcq_A
58,1ec2_A,1hpv_A
59,1ec2_A,1hpv_B


#### 6.4.3 Delete Row if Protein 1 Value == Protein 2 Value

In [165]:
df_seq_matching_filtered = df_seq_matching_filtered[df_seq_matching_filtered['Protein 1'] != df_seq_matching_filtered['Protein 2']]
df_seq_matching_filtered.head()

,Protein 1,Protein 2
1,1ec2_A,1ec2_B
40,1ec2_A,7wcq_A
58,1ec2_A,1hpv_A
59,1ec2_A,1hpv_B
88,1ec2_A,1u8g_A


### 6.5 Save Seq Matching Filtered DF

In [158]:
df_seq_matching_filtered.to_csv("./Data HIV Type 1/seq_matching_result_filtered_50_new_1.tsv", sep="\t", index=False)

## 7. Add "Cluster Information" to HIV 1 Filtered Original Dataset

In [359]:
original_hiv_filtered_df = pd.read_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded-hiv-only-n-query.csv")
original_hiv_filtered_df = original_hiv_filtered_df.drop("Unnamed: 0", axis=1)
original_hiv_filtered_df["cluster"] = None
original_hiv_filtered_df.head()

,C_ID,Name,Common Name,Scientific Name,Quantity,Non plant ingredient,Action,Application,Comments,Bacteria,...,InChIKey,InChICode,SMILES,target_name,species,bdb_id,ligand_smiles,pdb_link,query,cluster
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Protease,(Human immunodeficiency virus type 1),BDBM50478896,C[C@H]1[C@@H]2CC[C@](C)(O)[C@@H]3[C@H]4[C@@H]5...,https://www.rcsb.org/search?request=%7B%22quer...,"9HVP,7YF6,7WCQ,7WBS,6OYR,6OYD,6OGP,6MCS,6MCR,5...",None
1,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Gag-Pol polyprotein [489-587],(Human immunodeficiency virus type 1),BDBM50613450,CC(=C)[C@H]1CC[C@@]2(C)[C@H]1CC[C@]1(C)[C@@H]2...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",None
2,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Glycogen phosphorylase,(Human immunodeficiency virus type 1),BDBM50050957,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",None
3,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,muscle form,(Human immunodeficiency virus type 1),BDBM50050952,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",None
4,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Protein kinase C epsilon type,(Human immunodeficiency virus type 1),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",None


In [219]:
cluster_df = pd.read_excel("./Data HIV Type 1/cluster.xlsx")
cluster_df.head()

,Cluster,Sequence,Category
0,1,"3v6d_A, 3v6d_B, 3v6d_C, 3v6d_D, 8dx3_A, 5vqu_A...","HIV Reverse Transcriptase (RT), Seq Protease"
1,2,"1ec2_A, 1ec2_B, 7wcq_A, 1hpv_A, 1hpv_B, 1u8g_A...",HIV Protease
2,3,"5fuu_A, 5fuu_C, 5fuu_E, 6vpx_A, 6vpx_B, 6vpx_C...",HIV Envelope Glycoproteins
3,4,"5fuu_B, 5fuu_D, 5fuu_F, 6vpx_D, 6vpx_E, 6vpx_F...",HIV Envelope Glycoproteins
4,5,"7urn_A, 7urn_B, 7urn_C, 7urn_D, 7urn_E, 7urn_F...",HIV Capsid


In [415]:
a = original_hiv_filtered_df.iloc[5]
b = cluster_df.iloc[9]

In [411]:
protein_code_cid = a["query"].split(",")
print(protein_code_cid)

['5O2U', '1L6N', '7ASL', '7ASH', '6SMU', '6SKN', '6SKK', '9DWD', '9D6C', '6SLU', '6SLQ', '6SKM', '9CWV', '8EJL', '7URN', '7M9F', '6X63']


In [412]:
protein_code_cluster_1 = b["Sequence"].split(", ")
protein_code_cluster_1= [item[:4].upper() for item in protein_code_cluster_1] 
protein_code_cluster_1 = set(protein_code_cluster_1)
print(protein_code_cluster_1)

{'1RTD'}


In [413]:
matching_items = list(set(protein_code_cluster_1) & set(protein_code_cid))
len(matching_items)

0

In [416]:

original_hiv_filtered_df["cluster"] = original_hiv_filtered_df["cluster"].apply(lambda x: [] if x is None else x)

for index_1, row_1 in original_hiv_filtered_df.iterrows():
    query = set(row_1["query"].split(','))
    c_id = row_1["C_ID"]
    clusters_to_add = []
    # print(f"{c_id} = {query}")
    
    for index_2, row_2 in cluster_df.iterrows():
        sequence = row_2["Sequence"].split(', ') 
        sequence = [item[:4].upper() for item in sequence] 
        sequence = set(sequence)
        cluster = row_2["Cluster"]
        
        matching_items = list(set(sequence) & set(query))
        
        if sequence & query:
            clusters_to_add.append(cluster)
        
        # if matching_items:
        #     if row_1["cluster"]:
        #         row_1["cluster"] += cluster
        #     else:
        #         row_1["cluster"] = cluster
    original_hiv_filtered_df.at[index_1, "cluster"] = ", ".join(map(str, clusters_to_add)) if clusters_to_add else pd.NA

In [417]:
original_hiv_filtered_df.head()

,C_ID,Name,Common Name,Scientific Name,Quantity,Non plant ingredient,Action,Application,Comments,Bacteria,...,InChIKey,InChICode,SMILES,target_name,species,bdb_id,ligand_smiles,pdb_link,query,cluster
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...","Raisin, Grape, Purging Cassia, Long Melon, Cuc...","Vitis vinifera, Cassia fistula, Cucumis melo, ...","20gm, 10gm, 25gm, 380gm, 80gm, 100gm, 50gm","Salt, Sodium chioride: 10gm; Sea Shell, Gallip...","Curative curbancle, diarrhoea,Blood poisoning,...",NaN,NaN,"0.0, 1.0",...,IXORZMNAPKEEDV-BKYUDGNBNA-N,"InChI=1S/C19H22O6/c1-9-7-17-8-18(9,24)5-3-10(1...",C=C1C[C@]23C[C@@]1(O)CC[C@H]2[C@@]12C=C[C@H](O...,Protease,(Human immunodeficiency virus type 1),BDBM50478896,C[C@H]1[C@@H]2CC[C@](C)(O)[C@@H]3[C@H]4[C@@H]5...,https://www.rcsb.org/search?request=%7B%22quer...,"9HVP,7YF6,7WCQ,7WBS,6OYR,6OYD,6OGP,6MCS,6MCR,5...",2
1,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Gag-Pol polyprotein [489-587],(Human immunodeficiency virus type 1),BDBM50613450,CC(=C)[C@H]1CC[C@@]2(C)[C@H]1CC[C@]1(C)[C@@H]2...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",2
2,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Glycogen phosphorylase,(Human immunodeficiency virus type 1),BDBM50050957,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",2
3,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,muscle form,(Human immunodeficiency virus type 1),BDBM50050952,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",2
4,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...","Long Melon, Cucumber seed, Paddy, Bishop's Wee...","Cucumis melo, Cucumis sativus, Oryza sativa, T...","25gm, 380gm, 10gm, 40gm, 5gm, 80gm, 90gm, 100g...","Cow's Milk, 12L; Sugar, 9.6kg; , Rock Salt, So...","Curative for inflammation, strangury, Painful ...",NaN,NaN,"0.0, 1.0",...,RSQSQJNRHICNNH-DZBXMYAANA-N,InChI=1S/C19H24O5/c1-9-7-18-8-10(9)3-4-11(18)1...,C=C1C[C@]23C[C@H]1CC[C@H]2[C@@]12CC[C@H](O)[C@...,Protein kinase C epsilon type,(Human immunodeficiency virus type 1),BDBM50050955,CC(=C)[C@@H]1CC[C@@]2(CC[C@]3(C)[C@H](CC[C@@H]...,https://www.rcsb.org/search?request=%7B%22quer...,"1ODW,1HXB,1HIV,1W5Y,1W5X,1W5W,1W5V,1MES,1MER,1...",2


In [418]:
original_hiv_filtered_df.to_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded-hiv-only-n-query-cluster.csv")

## 8. Cluster, C_ID, Ayurvedic Formula

In [423]:
original_hiv_cluster_df = pd.read_csv("./Data HIV Type 1/ayurvedic-knapsack-grouped-bindingdb-cleaner-exploded-hiv-only-n-query-cluster.csv")
original_hiv_cluster_df.dropna(subset=["cluster"], inplace=True)
original_hiv_cluster_df = original_hiv_cluster_df[['C_ID', 'Name', 'cluster']]
original_hiv_cluster_df.head()

,C_ID,Name,cluster
0,"C00000003,","Anantadikwath (240gm), Drakshadi Kwath (40gm),...",2
1,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...",2
2,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...",2
3,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...",2
4,"C00000004,","Kushabaleha (5kg), Sukrasanjibani Modak (15.71...",2


In [462]:
data = []
for index, row in original_hiv_cluster_df.iterrows():
    cluster_list = row["cluster"].split(", ")  # Simpan sebagai list
    C_ID = row["C_ID"].strip(",")
    ayurvedic_formula = row["Name"]
    
    for cluster in cluster_list:  # Loop setiap elemen dalam list
        data.append([cluster, C_ID, ayurvedic_formula])  # Tambahkan elemen individu
    
cluster_cid_name_df = pd.DataFrame(data, columns=["Cluster", "C_ID", "Name"])
cluster_cid_name_df = cluster_cid_name_df.sort_values(by="Cluster", ascending=True)

cluster_cid_name_df.head()

,Cluster,C_ID,Name
3289,1,C00013328,"Kutajadi Kwath (90gm), Pijushballi Ras (280gm)..."
4361,1,C00026107,"Kataphaladi Kwath (190gm), Chandanadi Kwath (1..."
1818,1,C00004082,Bamanamirita Ras (110gm)
4358,1,C00026095,"Kataphaladi Kwath (190gm), Chandanadi Kwath (1..."
1821,1,C00004089,"Shri Madanananda Modak (4.36kg), Kosturibhusha..."


In [463]:
cluster_cid_name_df = cluster_cid_name_df.drop_duplicates(subset=["Cluster", "C_ID", "Name"], keep="first")
cluster_cid_name_df.head()

,Cluster,C_ID,Name
3289,1,C00013328,"Kutajadi Kwath (90gm), Pijushballi Ras (280gm)..."
4361,1,C00026107,"Kataphaladi Kwath (190gm), Chandanadi Kwath (1..."
1818,1,C00004082,Bamanamirita Ras (110gm)
4358,1,C00026095,"Kataphaladi Kwath (190gm), Chandanadi Kwath (1..."
1821,1,C00004089,"Shri Madanananda Modak (4.36kg), Kosturibhusha..."


In [464]:
cluster_cid_name_df = cluster_cid_name_df.groupby("Cluster", as_index=False).agg({
    "C_ID": lambda x: ', '.join(x),   # Gabungkan C_ID dengan koma
    "Name": lambda x: '; '.join(x)    # Gabungkan Name dengan titik koma
})

cluster_cid_name_df["Cluster"] = cluster_cid_name_df["Cluster"].astype(int)
cluster_cid_name_df = cluster_cid_name_df.sort_values(by="Cluster", ascending=True)

cluster_cid_name_df.to_excel("./Data HIV Type 1/cluster_cid_ayurvedic_formula.xlsx", index=False)

cluster_cid_name_df.head()

,Cluster,C_ID,Name
0,1,"C00013328, C00026107, C00004082, C00026095, C0...","Kutajadi Kwath (90gm), Pijushballi Ras (280gm)..."
2,2,"C00029493, C00061416, C00029451, C00049970, C0...","Kshudabati Bati (2.304kg), Kshudhabati Gudika ..."
3,3,"C00029493, C00061416, C00000013, C00029724, C0...","Kshudabati Bati (2.304kg), Kshudhabati Gudika ..."
4,4,"C00029724, C00000012, C00000018, C00029493, C0...",Somnath Ras ?250gm); Sukrasanjibani Modak (15....
5,5,"C00000029, C00000009, C00000034, C00000027, C0...","Sukrasanjibani Modak (15.71kg), Bamanamirita R..."
